In [ ]:
import mediapipe as mp
import cv2
import os
from os.path import join, isdir
import pandas as pd
import numpy as np
from IPython.display import clear_output 

In [ ]:
# https://www.kaggle.com/datasets/kapillondhe/american-sign-language


ROOT = "/Users/vaisakh/Desktop/last-research/ASL_Dataset/ASL_Train"
EXT = ".jpg"
CSV_PATH = "/Users/vaisakh/research_final_codebase/dataset.csv"

In [ ]:
img_paths = []
for label in os.listdir(ROOT):
    if not isdir(join(ROOT, label)): continue;
    for image in os.listdir(join(ROOT, label)):
        img_path = join(ROOT, label, image)
        if not img_path.endswith(EXT): continue
        img_paths.append([label, img_path])

In [ ]:
len(img_paths)

In [ ]:
cols = [
    '0x', '0y', '1x', '1y', '2x', '2y', '3x', '3y', 
    '4x', '4y', '5x', '5y', '6x', '6y', '7x', '7y', 
    '8x', '8y', '9x', '9y', '10x', '10y', '11x', '11y', 
    '12x', '12y', '13x', '13y', '14x', '14y', '15x', 
    '15y', '16x', '16y', '17x', '17y', '18x', '18y', 
    '19x', '19y', '20x', '20y', 'label']

df = pd.DataFrame(columns=cols)

In [ ]:
def pre_process(landmark):
    processed_landmarks = []
    relative_landmarks= []
    for i in landmark[0].landmark:
        processed_landmarks.append([i.x, i.y])

    for i in processed_landmarks:
        relative_landmarks.append([
            i[0]-processed_landmarks[0][0],
            i[1]-processed_landmarks[0][1],
        ])
    
    flatten = []
    for i in relative_landmarks:
        flatten.append(i[0])
        flatten.append(i[1])
    
    maxVal = max(list(map(abs, flatten)))
    def norm(n): return n/maxVal
    normalised = list(map(norm, flatten))

    return normalised

In [74]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) 
count= 0
corrupted_files = []

for label, image_path in img_paths[:100]:
    count+=1
    clear_output()
    print(count, img)

    image = cv2.imread(image_path)
    try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = hands.process(image)
        if not result.multi_hand_landmarks: continue

        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        for hand in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),)
        cv2.imshow("wind", image)
        cv2.waitKey(0)

        preprocessed = pre_process(result.multi_hand_landmarks)
        preprocessed.append(label)
        df.loc[len(df)] = preprocessed
    except:
        corrupted_files.append(image_path)
        print("corrupted:", image_path)

    

    
cv2.destroyAllWindows()

100 /Users/vaisakh/Desktop/last-research/ASL_Dataset/ASL_Train/R/1446.jpg


/Users/vaisakh/miniforge3/lib/python3.10/site-packages/google/protobuf/symbol_database.py:78: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [75]:
df.head()

,0x,0y,1x,1y,2x,2y,3x,3y,4x,4y,...,16y,17x,17y,18x,18y,19x,19y,20x,20y,label
0,0.0,0.0,-0.137939,-0.102408,-0.210286,-0.290843,-0.136485,-0.442677,-0.012725,-0.526258,...,-0.329419,0.175385,-0.360373,0.113739,-0.445154,0.046301,-0.335742,0.014966,-0.240969,R
1,0.0,0.0,-0.137939,-0.102408,-0.210286,-0.290843,-0.136485,-0.442677,-0.012725,-0.526258,...,-0.329419,0.175385,-0.360373,0.113739,-0.445154,0.046301,-0.335742,0.014966,-0.240969,R
2,0.0,0.0,-0.145031,-0.104455,-0.217983,-0.292274,-0.141535,-0.448530,-0.016454,-0.531183,...,-0.323900,0.172319,-0.365221,0.118423,-0.452070,0.046924,-0.347243,0.001998,-0.252909,R
3,0.0,0.0,-0.136677,-0.163675,-0.182776,-0.361404,-0.095506,-0.515515,0.029945,-0.599974,...,-0.393487,0.098828,-0.450658,0.092487,-0.562930,0.070438,-0.509285,0.052513,-0.442569,U
4,0.0,0.0,-0.128625,-0.128503,-0.183046,-0.311779,-0.074872,-0.419089,0.055082,-0.430645,...,-0.384004,0.212421,-0.348918,0.149815,-0.425931,0.068590,-0.362978,0.031880,-0.300151,U


In [76]:
df.to_csv(CSV_PATH)
df['label'].value_counts()

U    191
R      5
Name: label, dtype: int64

In [77]:
print(corrupted_files)

['/Users/vaisakh/Desktop/last-research/ASL_Dataset/ASL_Train/R/1446.jpg']
